In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from scipy import stats
from IPython.display import display, HTML

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing
import scipy.io
from scipy.spatial.distance import pdist, squareform #scipy spatial distance
import sklearn as sk
import sklearn.metrics.pairwise
import matplotlib.pyplot as plt
import os
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, LeakyReLU
from keras import metrics
from keras import backend as K
import time
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.utils import np_utils

Using TensorFlow backend.
C:\Users\NONAME\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\NONAME\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\NONAME\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\NONAME\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Reshape
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split

In [3]:
mat = scipy.io.loadmat('../Dataset/EMG/DL_all_subject.mat')
EMG_data=mat['DL_all_subject']

In [4]:
def create_segments_and_labels(df, time_steps, step, label_name):

    # Ch1, Ch2 data as features
    N_FEATURES = 2
    # Number of steps to advance in each iteration (for me, it should always
    # be equal to the time_steps in order to have no overlap between segments)
    # step = time_steps
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        xs = df['Ch1'].values[i: i + time_steps]
        ys = df['Ch2'].values[i: i + time_steps]
        # Retrieve the most often used label in this segment
        label = stats.mode(df[label_name][i: i + time_steps])[0][0]
        segments.append([xs, ys])
        labels.append(label)

    # Bring the segments into a better shape
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    labels = np.asarray(labels)

    return reshaped_segments, labels



In [5]:
df = pd.DataFrame({'label': EMG_data[:, 0], 'Ch1': EMG_data[:, 1], 'Ch2': EMG_data[:, 2]})

In [6]:
df.head()
label = df['label']
df_v1 = df[['Ch1', 'Ch2']]
df.head()

,label,Ch1,Ch2
0,0.0,0.072198,0.202185
1,0.0,0.148703,0.227693
2,0.0,0.148703,0.304216
3,0.0,0.021195,0.227693
4,0.0,0.352716,0.278708


In [7]:
Time_periods = 150
step_distance = 15

X, Y = create_segments_and_labels(df, Time_periods, step_distance, 'label')

In [8]:
msk = np.random.rand(len(X)) < 0.8
X_train = X[msk]
Y_train = Y[msk]
X_test = X[~msk]
Y_test = Y[~msk]

In [9]:
X_train.shape
idx = np.random.permutation(len(X_train))
X_train = X_train[idx]
Y_train = Y_train[idx]

In [10]:
input_shape = (Time_periods*2)
X_train = X_train.reshape(X_train.shape[0], input_shape)
print('x_train shape:', X_train.shape)
print('input_shape:', input_shape)
Y_train.shape

x_train shape: (143995, 300)
input_shape: 300


(143995,)

In [11]:
x_train = X_train.astype('float32')
y_train = Y_train.astype('float32')
y_train_hot = np_utils.to_categorical(y_train, 6)
print('New y_train shape: ', y_train_hot.shape)


New y_train shape:  (143995, 6)


In [12]:
model_m = Sequential()
model_m.add(Reshape((150, 2), input_shape=(input_shape,)))
model_m.add(Conv1D(100, 10, activation='relu', input_shape=(Time_periods, 2)))
model_m.add(Conv1D(100, 10, activation='relu'))
model_m.add(MaxPooling1D(3))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(Conv1D(160, 10, activation='relu'))
model_m.add(GlobalAveragePooling1D())
model_m.add(Dropout(0.5))
model_m.add(Dense(6, activation='softmax'))
print(model_m.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 150, 2)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 141, 100)          2100      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 132, 100)          100100    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 44, 100)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 35, 160)           160160    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 26, 160)           256160    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 160)               0         
__________

In [13]:
model_m.compile(loss='categorical_crossentropy',
                optimizer='adam', metrics=['accuracy'])

# Hyper-parameters
BATCH_SIZE = 32
EPOCHS = 10

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
history = model_m.fit(x_train,
                      y_train_hot,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      validation_split=0.2,
                      verbose=1)

Train on 115196 samples, validate on 28799 samples
Epoch 1/10
115196/115196 [==============================] - 397s 3ms/step - loss: 0.9425 - acc: 0.6121 - val_loss: 0.6601 - val_acc: 0.7241
Epoch 2/10
115196/115196 [==============================] - 359s 3ms/step - loss: 0.6120 - acc: 0.7534 - val_loss: 0.4474 - val_acc: 0.8305
Epoch 3/10
115196/115196 [==============================] - 339s 3ms/step - loss: 0.4634 - acc: 0.8224 - val_loss: 0.3646 - val_acc: 0.8629
Epoch 4/10
115196/115196 [==============================] - 354s 3ms/step - loss: 0.3831 - acc: 0.8584 - val_loss: 0.3142 - val_acc: 0.8828
Epoch 5/10
115196/115196 [==============================] - 369s 3ms/step - loss: 0.3276 - acc: 0.8809 - val_loss: 0.2977 - val_acc: 0.8864
Epoch 6/10
115196/115196 [==============================] - 416s 4ms/step - loss: 0.2843 - acc: 0.8969 - val_loss: 0.2552 - val_acc: 0.9087
Epoch 7/10
115196/115196 [==============================] - 405s 4ms/step - loss: 0.2560 - acc: 0.9073 - val_

In [14]:
input_shape = (Time_periods*2)
X_test = X_test.reshape(X_test.shape[0], input_shape)
y_pred_test = model_m.predict(X_test)
# Take the class with the highest probability from the test predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_pred_test.shape
print(max_y_pred_test)
Y_test = Y_test.astype('int')
print(Y_test)

[0 0 0 ... 5 5 5]
[0 0 0 ... 5 5 5]


In [15]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(Y_test, max_y_pred_test)
print(classification_report(Y_test, max_y_pred_test))


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      6029
           1       0.94      0.95      0.94      6130
           2       0.98      0.92      0.95      5968
           3       0.91      0.93      0.92      5972
           4       0.90      0.96      0.93      5962
           5       0.95      0.87      0.91      5934

   micro avg       0.93      0.93      0.93     35995
   macro avg       0.93      0.93      0.93     35995
weighted avg       0.93      0.93      0.93     35995



In [16]:
print(accuracy)

0.9314904847895541
